In [1]:
import ray


In [3]:
class SimpleClass:
    def __init__(self, value):
        self.value = value

    def increase(self):
        self.value += 1
        print(self.value)
ray.init()      
SimpleClass = ray.remote(SimpleClass)
obj = SimpleClass.remote(2)
obj.increase.remote()
getattr(obj, "increase").remote()
ray.shutdown()

ObjectRef(63964fa4841d4a2e59229e3d057d393869f45ab50100000001000000)

(SimpleClass pid=98259) 3


In [ ]:
from fedrec.multiprocessing.process_manager import RayProcessManager
rpm  = RayProcessManager()
